<a href="https://colab.research.google.com/github/ryu0315360/Testing/blob/main/ML_augmetation_duration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# library & class define

In [ ]:
!pip install keras.utils

  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2656 sha256=aa65f257c65e234392f8f28ba04c588c3752d96a43ac9479bb565f0bdaa0139e
  Stored in directory: /root/.cache/pip/wheels/d0/dd/3b/493952a5240d486a83805d65360dedadbadeae71d25e2c877f
Successfully built keras.utils


In [ ]:
# Keras
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
from keras import losses, models, optimizers
from keras.activations import relu, softmax
from keras.layers import (Convolution2D, GlobalAveragePooling2D, BatchNormalization, Flatten, Dropout,
                          GlobalMaxPool2D, MaxPool2D, concatenate, Activation, Input, Dense)

# sklearn
from sklearn.preprocessing import LabelEncoder

# Other  
from tqdm import tqdm, tqdm_pandas
import scipy
from scipy.stats import skew

import json
import tensorflow as tf
from matplotlib.pyplot import specgram

import IPython.display as ipd  # To play sound in the notebook

In [ ]:
import pandas as pd
import librosa
import librosa.display
import soundfile
import os, glob, pickle, sys
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
# from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

In [ ]:
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
## path!

from google.colab import drive
drive.mount('/content/gdrive')

path_orig = 'gdrive/MyDrive/data/audio_speech_actors_01-24/Actor_*/*.wav' ## RAVDESS before padding 
path_tess = 'gdrive/MyDrive/TESS_Data/dataverse_files/*.wav'  ## tess before padding

Mounted at /content/gdrive


In [ ]:
## from 2sec data (after duration processing)

path_orig2 = 'gdrive/MyDrive/data/audio_speech_actors_01-24/2sec_data/*.wav' ## RAVDESS before padding 
path_tess2 = 'gdrive/MyDrive/TESS_Data/2sec_data/*.wav' 

In [ ]:
pip install pydub

In [ ]:
pip install audiomentations

In [ ]:
pip install audioread

In [ ]:
from pydub import AudioSegment

In [ ]:
import tensorflow.keras.layers as tfl

In [ ]:
import random
import numpy as np
import tensorflow as tf
random.seed(299)
np.random.seed(299)
tf.random.set_seed(299)

In [ ]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
# Emotions to observe
# observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
emotions_tess = {
  'neutral':'neutral',
  # '02':'calm',
  'happy':'happy',
  'sad':'sad',
  'angry':'angry',
  'fear':'fearful',
  'disgust':'disgust',
  # '08':'surprised'
}

# augmentation

In [ ]:
## Augmentation (X3) # 13 min 31 sec
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import numpy as np
import soundfile as sf

# augment = Compose([
#     # AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
#     TimeStretch(min_rate=0.8, max_rate=1.25, p=1),
#     # PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
#     Shift(min_fraction=-0.5, max_fraction=0.5, p=0.5),
# ])

# Generate 2 seconds of dummy audio for the sake of example
# samples = np.random.uniform(low=-0.2, high=0.2, size=(32000,)).astype(np.float32)

# print(samples)

# Augment/transform/perturb the audio data
# augmented_samples = augment(samples=samples, sample_rate=16000)

augment_stretch = Compose([
    TimeStretch(min_rate=0.8, max_rate=1.25, p=1)
])

augment_noise = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=1)
])

for file in glob.glob(path_tess):

  try:
    file_name = os.path.basename(file)
    y, sr = librosa.load(file)
  
  except:
    print("decoding error")
    continue

  # for i in range(3):
  augmented_samples = augment_stretch(samples=y, sample_rate=sr)
  name = str(file)[:-4] + "_augS_" + str(i+1) + '.wav' 
  # name = "aug_" + str(i+1) + "_" + str(file)
  print(name)
  sf.write(name, augmented_samples, sr)

  augmented_samples = augment_noise(samples=y, sample_rate=sr)
  name = str(file)[:-4] + "_augN_" + str(i+1) + '.wav' 
  # name = "aug_" + str(i+1) + "_" + str(file)
  print(name)
  sf.write(name, augmented_samples, sr)

for file in glob.glob(path_orig):

  try:
    file_name = os.path.basename(file)
    y, sr = librosa.load(file)
  
  except:
    print("decoding error")
    continue

  # for i in range(3):
  augmented_samples = augment_stretch(samples=y, sample_rate=sr)
  name = str(file)[:-4] + "_augS_" + str(i+1) + '.wav' 
  # name = "aug_" + str(i+1) + "_" + str(file)
  print(name)
  sf.write(name, augmented_samples, sr)

  augmented_samples = augment_noise(samples=y, sample_rate=sr)
  name = str(file)[:-4] + "_augN_" + str(i+1) + '.wav' 
  # name = "aug_" + str(i+1) + "_" + str(file)
  print(name)
  sf.write(name, augmented_samples, sr)


# duration setting

In [ ]:
def check_duration_error():
  cnt = 0
  for file in glob.glob(path_tess2):
    cnt += 1
    sound = AudioSegment.from_wav(file)
    sound = sound.set_channels(1)

    if sound.duration_seconds != float(2.0):
      print("duration error")
      print(file)
      print(sound.duration_seconds)

  print(cnt)


check_duration_error()

In [ ]:
def padding_duration():
  for file in glob.glob(path_tess):
    try:
      file_name=os.path.basename(file)

      if file_name.split('_')[2] == 'ps.wav':
        print("ps")
        continue
      # y, sr = librosa.load(file_name, offset=1.0, duration=3.0)
      sound = AudioSegment.from_wav(file)
      sound = sound.set_channels(1)
    except:
      print("Decoding error")
      continue

    pad_ms = 2000  # milliseconds of silence needed # 3sec

    #print("before duration: ", sound.duration_seconds)

    if float(sound.duration_seconds) < float(2.0): 
      print("shorter than 2")
      silence = AudioSegment.silent(duration = 1000)
      # print("temp: ", temp)
      print("sound duration: ",sound.duration_seconds)
      # audio = AudioSegment.from_wav(file)
      sound = silence + sound + silence  # Adding silence after the audio
      mid_duration = int((sound.duration_seconds/2)*1000)
      sound = sound[mid_duration - 1000:mid_duration+1000]
      print("new sound duration: ", sound.duration_seconds)
      new_filename = '/'.join(file.split('/')[:-2]) + "/2sec_data/" + '/'.join(file.split('/')[-1:])
      # print(new_filename)
      sound.export(new_filename, format='wav')
    else:
      print("longer than 2")
      mid_duration = int((sound.duration_seconds/2)*1000)
      print(mid_duration)
      sound = sound[mid_duration - 1000:mid_duration +1000]
      new_filename = '/'.join(file.split('/')[:-2]) + "/2sec_data/" + '/'.join(file.split('/')[-1:])
      print("new sound duration: ", sound.duration_seconds)
      # print(new_filename)
      sound.export(new_filename, format="wav") ## exporting to new folder!!
    
    #print("after duration: ", sound.duration_seconds)

    if sound.duration_seconds != 2.0:
      print("duration error")
      return 0

  for file in glob.glob(path_orig):
    try:
      file_name=os.path.basename(file)
      print(file)

      # y, sr = librosa.load(file_name, offset=1.0, duration=3.0)
      sound = AudioSegment.from_wav(file)
      sound = sound.set_channels(1)
    except:
      print("Decoding error")
      continue

    pad_ms = 2000  # milliseconds of silence needed # 3sec

    #print("before duration: ", sound.duration_seconds)

    if float(sound.duration_seconds) < float(2.0): 
      print("shorter than 2")

      silence = AudioSegment.silent(duration = 1000)
      # print("temp: ", temp)
      print("sound duration: ",sound.duration_seconds)
      # audio = AudioSegment.from_wav(file)
      sound = silence + sound + silence  # Adding silence after the audio
      mid_duration = int((sound.duration_seconds/2)*1000)
      sound = sound[mid_duration - 1000:mid_duration+1000]
      print("new sound duration: ", sound.duration_seconds)
      new_filename = '/'.join(file.split('/')[:-2]) + "/2sec_data/" + '/'.join(file.split('/')[-1:])
      # print(new_filename)
      sound.export(new_filename, format='wav')
    else:
      print("longer than 2")
      mid_duration = int((sound.duration_seconds/2)*1000)
      print(mid_duration)
      sound = sound[mid_duration - 1000:mid_duration +1000]
      new_filename = '/'.join(file.split('/')[:-2]) + "/2sec_data/" + '/'.join(file.split('/')[-1:])
      print("new sound duration: ", sound.duration_seconds)
      print(new_filename)
      try:
        sound.export(new_filename, format="wav")
      except:
        print('export 안됨')
        continue
    
    #print("after duration: ", sound.duration_seconds)

    if sound.duration_seconds != 2.0:
      print("duration error")
      return 0

  

  return 0

padding_duration()